In [168]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [169]:
df_train = pd.read_csv('GUIDE_Train.csv',nrows=2000000)

C:\Users\New\AppData\Local\Temp\ipykernel_20540\2714306493.py:1: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('GUIDE_Train.csv',nrows=2000000)


In [189]:
df_train.head(5)

,id,incidentid,alertid,orgid,detectorid,alerttitle,category,entitytype,evidencerole,deviceid,sha256,ipaddress,url,accountsid,accountupn,accountobjectid,accountname,devicename,networkmessageid,registrykey,registryvaluename,registryvaluedata,applicationid,applicationname,oauthapplicationid,filename,folderpath,resourceidname,osfamily,osversion,countrycode,state,city,year,month,day,hour,minute,incidentgrade
0,180388628218,612,123247,0,7,6,10,13,1,98799,138268,27,160396,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,31,6,3,2024,6,4,6,5,2
1,455266534868,326,210035,88,58,43,7,28,0,98799,138268,360606,160396,22406,23032,22795,24887,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630,2024,6,14,3,1,1
2,1056561957389,58352,712507,809,423,298,10,27,1,98799,138268,360606,68652,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630,2024,6,13,4,52,1
3,1279900258736,32992,774301,92,2,2,1,27,1,98799,138268,360606,13,441377,673934,425863,453297,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630,2024,6,10,16,39,0
4,214748368522,4359,188041,148,9,74,6,28,0,98799,138268,360606,160396,449,592,440,479,153085,529644,1631,635,860,2251,3421,881,289573,117668,3586,5,66,242,1445,10630,2024,6,15,1,8,2


In [192]:
df_train['state'].value_counts()

state
1445    1856041
0         21916
1          8555
2          6214
3          4805
4          4063
5          3258
6          3171
7          3095
8          2656
9          2539
11         2530
12         2461
10         2398
13         2138
15         1943
14         1899
21         1596
17         1557
18         1507
16         1441
19         1420
20         1399
23         1387
25         1336
22         1324
24         1248
27         1109
29         1076
26         1043
28         1020
32          937
30          884
31          839
34          801
36          791
35          768
37          754
38          750
33          741
40          691
42          661
39          642
43          637
41          633
46          616
50          600
49          571
45          547
47          546
44          537
48          514
51          505
53          437
54          411
57          401
52          393
56          390
61          375
59          369
62          362
55          361
60

In [170]:
df_train.columns = df_train.columns.str.lower()

df_train = df_train.drop(columns=['resourcetype',
 'actiongrouped',
 'actiongranular',
 'threatfamily',
 'emailclusterid',
 'antispamdirection',
 'roles',
 'suspicionlevel',
 'lastverdict',
 'mitretechniques'])

df_train = df_train.dropna()

In [171]:
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])

df_train['year'] = df_train['timestamp'].dt.year
df_train['month'] = df_train['timestamp'].dt.month
df_train['day'] = df_train['timestamp'].dt.day
df_train['hour'] = df_train['timestamp'].dt.hour
df_train['minute'] = df_train['timestamp'].dt.minute

df_train.drop('timestamp', axis=1, inplace=True)

# Assign target to last index
clumn = ['id', 'incidentid', 'alertid','orgid', 'detectorid',
       'alerttitle', 'category', 'entitytype', 'evidencerole',
       'deviceid', 'sha256', 'ipaddress', 'url', 'accountsid', 'accountupn',
       'accountobjectid', 'accountname', 'devicename', 'networkmessageid',
       'registrykey', 'registryvaluename', 'registryvaluedata',
       'applicationid', 'applicationname', 'oauthapplicationid', 'filename',
       'folderpath', 'resourceidname', 'osfamily', 'osversion', 'countrycode',
       'state', 'city', 'year', 'month', 'day', 'hour', 'minute',
        'incidentgrade']

df_train = df_train[clumn]

In [172]:
numerical_columns = df_train.select_dtypes(include=[np.number]).columns

columns = numerical_columns
# Create a mask to filter out outliers
mask = pd.Series([True] * len(df_train))

for column in columns:
    Q1 = df_train[column].quantile(0.25)
    Q3 = df_train[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Update mask to identify non-outlier rows
    column_mask = (df_train[column] >= lower_bound) & (df_train[column] <= upper_bound)
    mask = mask & column_mask

# Filter the DataFrame to remove outliers
cleaned_df = df_train[mask].reset_index(drop=True)


C:\Users\New\AppData\Local\Temp\ipykernel_20540\3774901892.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cleaned_df = df_train[mask].reset_index(drop=True)


In [173]:
categorical_column = df_train.select_dtypes(include='object').columns

le = LabelEncoder()
for col in categorical_column:
    df_train[col] = le.fit_transform(df_train[col])

In [174]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

# Assuming your dataframe is called 'df' and the column is 'incidentgrade'

# Get the value counts
value_counts = df_train['incidentgrade'].value_counts()

# Find the minimum count
min_count = value_counts.min()

# Create a list to store the balanced dataframes
balanced_dfs = []

# Undersample each class
for class_value in value_counts.index:
    class_df = df_train[df_train['incidentgrade'] == class_value]

    if len(class_df) > min_count:
        # Undersample
        undersampled_df = resample(class_df,
                                   replace=False,    # sample without replacement
                                   n_samples=min_count,
                                   random_state=42)  # reproducible results
        balanced_dfs.append(undersampled_df)
    else:
        # If this class is already at or below the minimum, keep all samples
        balanced_dfs.append(class_df)

# Combine the balanced dataframes
df_balanced = pd.concat(balanced_dfs)

# Verify the new class distribution
print(df_balanced['incidentgrade'].value_counts())

incidentgrade
0    427359
2    427359
1    427359
Name: count, dtype: int64


In [175]:
df_balanced.shape

(1282077, 39)

In [176]:
X = df_balanced.iloc[:,:-1]
y = df_balanced.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)


In [158]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd

# Train a Random Forest model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Get feature importance
importances = model.feature_importances_
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

feature_importance_df


,Feature,Importance
1,incidentid,0.260836
3,orgid,0.210908
7,entitytype,0.106982
2,alertid,0.098270
35,day,0.081662
5,alerttitle,0.046378
0,id,0.044531
4,detectorid,0.039984
37,minute,0.031768
36,hour,0.030332


In [186]:
valid = pd.read_csv('test_v1.csv')

valid = valid.drop(columns=["Unnamed: 0"])

categorical_column = valid.select_dtypes(include='object').columns

le = LabelEncoder()
for col in categorical_column:
    valid[col] = le.fit_transform(valid[col])

X_test_ = valid.iloc[:1000,:-1]
y_test_ = valid.iloc[:1000,-1]

In [187]:
len(valid.columns)


39

random forest classifier

In [177]:

model = RandomForestClassifier()

model.fit(X_train, y_train)


RandomForestClassifier()

In [188]:
def evalv():
    from sklearn.metrics import f1_score, recall_score, confusion_matrix, accuracy_score

    # Predict on the test set
    y_pred_ = model.predict(X_test_)


    # Generate the Confusion Matrix
    conf_matrix = confusion_matrix(y_test_, y_pred_)

   
    print("Confusion Matrix:")
    print(conf_matrix)
    
    print(classification_report(y_test_, y_pred_))
evalv()

Confusion Matrix:
[[389  25  20]
 [ 20 179   7]
 [ 25  13 322]]
              precision    recall  f1-score   support

           0       0.90      0.90      0.90       434
           1       0.82      0.87      0.85       206
           2       0.92      0.89      0.91       360

    accuracy                           0.89      1000
   macro avg       0.88      0.89      0.88      1000
weighted avg       0.89      0.89      0.89      1000



LogisticRegression

In [43]:
model = LogisticRegression(class_weight='balanced')

model.fit(X_train,y_train)

evalv()

F1 Score: 0.3927622298836817
Recall: 0.39624391374913054
Accuracy: 0.39624391374913054
Confusion Matrix:
[[1860 1271 1182]
 [1424 1221 1668]
 [1111 1156 2046]]


GaussianNB

In [44]:
model = GaussianNB()
model.fit(X_train, y_train)
evalv()


F1 Score: 0.3454143430860879
Recall: 0.3460081922868846
Accuracy: 0.3460081922868846
Confusion Matrix:
[[1417 1422 1474]
 [1338 1386 1589]
 [1321 1318 1674]]


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train,y_train)

evalv()

F1 Score: 0.6158014824097023
Recall: 0.6143442306206044
Accuracy: 0.6143442306206044
Confusion Matrix:
[[2664 1001  648]
 [1226 2535  552]
 [1015  548 2750]]


In [49]:
# Define the RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# Define the parameter grid to search over
param_grid = {
    'n_estimators': [100, 200],        # Number of trees in the forest
    'max_depth': [10, 20, None],        # Maximum depth of each tree
    'min_samples_split': [2, 5],        # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2],          # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt'],       # Number of features to consider when looking for the best split
    'class_weight': [None, 'balanced']      # Class weighting to handle class imbalance
}
# Initialize GridSearchCV with the model and parameter grid
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2, scoring='f1')

# Fit the GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best parameters found: ", grid_search.best_params_)

# Print the best F1 score achieved during Grid Search
print("Best F1 Score achieved: ", grid_search.best_score_)

# Use the best estimator to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model's performance
print("Classification Report:")
print(classification_report(y_test, y_pred))



Fitting 5 folds for each of 96 candidates, totalling 480 fits


KeyboardInterrupt: 